In [80]:
import numpy as np
import shap
n_features = 'all'
term_pred = 'short_term'
data_file_name = 'data_y10'
endpt = 10
eval_times = 365.25*np.r_[1, np.arange(2, endpt+1, 2)]
work_dir = '/home/idies/workspace/Storage/hnguye78/persistent/CARDIA_project/cvd_outcome_rerun2'
work_dir = 'U:/Hieu/CARDIA_project/CARDIA_project/cvd_outcome_rerun_2'

In [81]:
# %pip install tensorflow
# %pip install lifelines

# !y | pip uninstall statsmodels 
# %pip install statsmodels==0.11.0

# %pip install keras

In [82]:
from __future__ import print_function
from tensorflow.keras import backend as K

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import matplotlib
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Dense, Dropout, Activation, LSTM, GRU, Embedding, Concatenate, Conv1D, GlobalMaxPooling1D, MaxPooling1D, GlobalAveragePooling1D, BatchNormalization, TimeDistributed
from tensorflow.keras import optimizers, layers, regularizers
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model
import math
from lifelines import KaplanMeierFitter
from lifelines import CoxPHFitter
from lifelines.utils import concordance_index
from sklearn.preprocessing import StandardScaler
from scipy import stats
import time


In [83]:
snippets_dir = '/home/idies/workspace/Storage/hnguye78/persistent/CARDIA_project/cvd_outcome_rerun'+ '/code/snippet'
snippets_dir = 'U:/Hieu/CARDIA_project/CARDIA_project/Git/Python_code/snippets'

import os, sys
sys.path.append(os.path.abspath(snippets_dir))
import nnet_survival
from cox import *


In [84]:
load_dir = work_dir+ '/csv_files'
# load data:
data_full = pd.read_csv(load_dir+'/'+data_file_name+'.csv')
data_full = data_full.select_dtypes(include =[np.number])


In [85]:
data_full

,ID,event,time,sex,race,cholesterol,hdl,hbp.medication,smoker,diabetes,...,C08MELAG_YEAR,C08SKNAG_YEAR,C08BRNAG_YEAR,C08STMAG_YEAR,C08OCAAG_YEAR,C08GALDA_YEAR,C08PRGAG_YEAR,C08KYSAG_YEAR,C08URNAG_YEAR,count_na
0,100016012504,0,3689.025,0,0,186.0,38.0,0,1,0,...,0,0,0,0,0,0,0,0,0,0.048673
1,100023004268,0,3689.025,0,0,166.0,43.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.055310
2,100033323702,0,3689.025,0,0,234.0,52.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.077434
3,100048415270,0,1169.000,0,1,137.0,40.0,0,1,0,...,0,0,0,0,0,0,0,0,18,0.008850
4,100056526386,0,3689.025,0,1,199.0,44.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.008850
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4309,416761219907,0,3689.025,1,0,225.0,57.0,0,0,0,...,0,0,0,0,0,0,14,0,0,0.048673
4310,416771521620,0,3689.025,0,1,204.0,76.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.004425
4311,416783315386,0,3689.025,1,0,159.0,58.0,0,0,0,...,0,0,0,0,0,0,8,0,0,0.112832
4312,416796224310,0,3689.025,1,1,164.0,55.0,0,0,0,...,0,0,0,0,0,0,14,0,0,0.070796


In [86]:
# load training id:
loaddir = work_dir+ '/csv_files'
trainingid_all = pd.read_csv(loaddir+'/all_training_ID_outerloop_cohort_0_10.csv')

## standardize feature space, then merge back to label space:
feature_space = data_full.drop(['ID','event','time'], axis = 1)
feature_name_list = feature_space.columns.values

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler().fit(feature_space)
scaled_feature_space = scaler.transform(feature_space)
scaled_feature_space_df = pd.DataFrame(data=scaled_feature_space[0:,0:])
scaled_feature_space_df.columns = feature_name_list
scaled_feature_space_df.insert(0, 'ID', data_full['ID'], True)
label = data_full.loc[:,['ID','event','time']]
data_full = pd.merge(label, scaled_feature_space_df)

In [87]:
data_full

,ID,event,time,sex,race,cholesterol,hdl,hbp.medication,smoker,diabetes,...,C08MELAG_YEAR,C08SKNAG_YEAR,C08BRNAG_YEAR,C08STMAG_YEAR,C08OCAAG_YEAR,C08GALDA_YEAR,C08PRGAG_YEAR,C08KYSAG_YEAR,C08URNAG_YEAR,count_na
0,100016012504,0,3689.025,-1.106215,-0.970306,0.235193,-1.090061,-0.126551,1.580754,-0.150863,...,-0.022243,-0.024272,-0.021537,-0.015227,-0.056678,-0.087086,-0.626333,-0.090552,-0.124130,0.773715
1,100023004268,0,3689.025,-1.106215,-0.970306,-0.356134,-0.732871,-0.126551,-0.632609,-0.150863,...,-0.022243,-0.024272,-0.021537,-0.015227,-0.056678,-0.087086,-0.626333,-0.090552,-0.124130,0.979473
2,100033323702,0,3689.025,-1.106215,-0.970306,1.654379,-0.089929,-0.126551,-0.632609,-0.150863,...,-0.022243,-0.024272,-0.021537,-0.015227,-0.056678,-0.087086,-0.626333,-0.090552,-0.124130,1.665334
3,100048415270,0,1169.000,-1.106215,1.030602,-1.213559,-0.947185,-0.126551,1.580754,-0.150863,...,-0.022243,-0.024272,-0.021537,-0.015227,-0.056678,-0.087086,-0.626333,-0.090552,8.591018,-0.460833
4,100056526386,0,3689.025,-1.106215,1.030602,0.619556,-0.661433,-0.126551,-0.632609,-0.150863,...,-0.022243,-0.024272,-0.021537,-0.015227,-0.056678,-0.087086,-0.626333,-0.090552,-0.124130,-0.460833
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4309,416761219907,0,3689.025,0.903984,-0.970306,1.388282,0.267261,-0.126551,-0.632609,-0.150863,...,-0.022243,-0.024272,-0.021537,-0.015227,-0.056678,-0.087086,1.968180,-0.090552,-0.124130,0.773715
4310,416771521620,0,3689.025,-1.106215,1.030602,0.767388,1.624583,-0.126551,-0.632609,-0.150863,...,-0.022243,-0.024272,-0.021537,-0.015227,-0.056678,-0.087086,-0.626333,-0.090552,-0.124130,-0.598005
4311,416783315386,0,3689.025,0.903984,-0.970306,-0.563099,0.338699,-0.126551,-0.632609,-0.150863,...,-0.022243,-0.024272,-0.021537,-0.015227,-0.056678,-0.087086,0.856246,-0.090552,-0.124130,2.762710
4312,416796224310,0,3689.025,0.903984,1.030602,-0.415267,0.124385,-0.126551,-0.632609,-0.150863,...,-0.022243,-0.024272,-0.021537,-0.015227,-0.056678,-0.087086,1.968180,-0.090552,-0.124130,1.459575


In [88]:
# decide on number of dicrete times:
#halflife=13.*365.25

#breaks = 365.25*np.r_[1, np.arange(2, endpt+1, 0.5)]
breaks = 365.25*np.arange(1, endpt,0.5)
#breaks=-np.log(1-np.arange(0.0,0.96,0.05))*halflife/np.log(2) 
n_intervals=len(breaks)-1
timegap = breaks[1:] - breaks[:-1]



In [89]:
# intialize hyperparam:
hidden_layers_sizes = 4
n_epochs = 50000
learning_rate = 0.001
lr_decay = 0.001
momentum = 0.9
n_in = data_full.shape[1]-3 # number of features


def nnet_pred_surv(y_pred, breaks, fu_time):
#Predicted survival probability from Nnet-survival model
#Inputs are Numpy arrays.
#y_pred: Rectangular array, each individual's conditional probability of surviving each time interval
#breaks: Break-points for time intervals used for Nnet-survival model, starting with 0
#fu_time: Follow-up time point at which predictions are needed
#
#Returns: predicted survival probability for each individual at specified follow-up time
  y_pred=np.cumprod(y_pred, axis=1)
  pred_surv = []
  for i in range(y_pred.shape[0]):
    pred_surv.append(np.interp(fu_time,breaks[1:],y_pred[i,:]))
  return np.array(pred_surv)

In [90]:

def f_nnet(X_df):
    # return model.predict([X[:,i] for i in range(X.shape[1])]).flatten()
    X = X_df.values
    y_hat=model.predict(X,verbose=0)
    last_yr_pred_prob_surv=np.cumprod(y_hat[:,0:np.nonzero(breaks>365*(endpt-1))[0][0]], axis=1)[:,-1]
    return last_yr_pred_prob_surv

def f_nnet_input_as_matrix(X):
    # return model.predict([X[:,i] for i in range(X.shape[1])]).flatten()
    y_hat=model.predict(X,verbose=0)
    last_yr_pred_prob_surv=np.cumprod(y_hat[:,0:np.nonzero(breaks>365*(endpt-1))[0][0]], axis=1)[:,-1]
    return last_yr_pred_prob_surv

def vimp_shap_tabular(feature_names, vals):
    feature_importance_df = pd.DataFrame(list(zip(feature_names, vals)), columns=['variable_name','vimp'])
    feature_importance_df.sort_values(by=['vimp'], ascending=False,inplace=True)
    feature_importance_df['normalized_vimp'] = (feature_importance_df['vimp'] - min(feature_importance_df['vimp']))/(max(feature_importance_df['vimp'])-min(feature_importance_df['vimp']))
    
    # merge with variable description:
    var_dict = pd.read_csv(load_dir+'/y5_all_vars_dictionary_manually_selecting_vars (3).csv', encoding= 'unicode_escape')
    var_dict[['Variable.Name', 'Variable.Label']]
    # merge vimp table with var dict to get var description:
    feature_importance_df_with_vardes = feature_importance_df.merge(
        var_dict[['Variable.Name', 'Variable.Label']], how = 'inner', right_on= 'Variable.Name', left_on = 'variable_name', copy= False)
    return(feature_importance_df_with_vardes[['Variable.Name','Variable.Label','normalized_vimp','vimp']])



In [91]:
# Outerloop:
for fold in range(25):

    print('')
    print('FOLD '+str(fold)+':')
    print('')

    trainingid = trainingid_all.iloc[:,fold]
    trainingid = trainingid[~np.isnan(trainingid)]
    eligible_id = data_full['ID'][data_full['ID'].isin(trainingid)]
    train_df = data_full.loc[data_full['ID'].isin(trainingid),:]
    del train_df['ID']
    test_df = data_full.loc[~data_full['ID'].isin(trainingid),:]
    del test_df['ID']


    # reformat the train and test set:
    y_train = nnet_survival.make_surv_array(train_df.time.values
                                            , train_df.event.values
                                            , breaks)
    y_test = nnet_survival.make_surv_array(test_df.time.values
                                           , test_df.event.values
                                           , breaks)
    featurespace_train_df = train_df.drop(['time','event'],axis =1)
    featurespace_test_df = test_df.drop(['time','event'],axis =1)

    featurespace_train = train_df.drop(['time','event'],axis =1).values
    featurespace_test = test_df.drop(['time','event'],axis =1).values



    ##########################
    # Build model:
    l2_final=0.01

    from numpy.random import seed

    seed(1)
    import tensorflow as tf
    import keras
    tf.random.set_seed(2)

    model = Sequential()
    model.add(Dense(hidden_layers_sizes
                    , input_dim=featurespace_train.shape[1]
                    , bias_initializer='zeros'
                    , kernel_regularizer=regularizers.l2(l2_final)))
    model.add(Activation('relu'))
    model.add(Dense(n_intervals))
    model.add(Activation('sigmoid'))
    # import keras.backend.tensorflow_backend as kk

    model.compile(loss=nnet_survival.surv_likelihood(n_intervals), optimizer=optimizers.RMSprop())
    early_stopping = EarlyStopping(monitor='loss', patience=20)
    history=model.fit(featurespace_train, y_train, batch_size=4063, epochs=100000
                      , callbacks=[early_stopping]
                      , verbose=0)


    # print loss of train and valid data:
    # print(model.evaluate(featurespace_train,y_train,verbose=0))
    # model.evaluate(featurespace_test,y_test,verbose=0)

    #Discrimination performance
    # y_pred is conditional prob of survival within each time interval
    y_pred_train = model.predict(featurespace_train,verbose=0)
    # cumprod = cumulative product, the probability of surviving from time 0 up to the time interval of interest
    # index -1 because we are interested in -1 
    last_yr_surv_train=np.cumprod(y_pred_train[:,0:np.nonzero(breaks>365*(endpt-1))[0][0]], axis=1)[:,-1]
    print('Train C-index fold', str(fold+1),':')
    print(concordance_index(train_df.time, last_yr_surv_train, train_df.event)) 

    y_pred=model.predict(featurespace_test,verbose=0)
    last_yr_surv=np.cumprod(y_pred[:,0:np.nonzero(breaks>365*(endpt-1))[0][0]], axis=1)[:,-1]
    print('Test C-index fold', str(fold+1),':')
    print(concordance_index(test_df.time,last_yr_surv, test_df.event))

    pred_surv = np.zeros((len(test_df.event), len(eval_times)))
    col=0
    for time in eval_times:
        pred_surv[:,col] = nnet_pred_surv(y_pred, breaks, time)
        col = col+1
    pred_surv = pd.DataFrame(data = pred_surv)
    # savedir = os.path.join(os.getcwd(),'python_files/csv_files/nnet_survival/all_features')
    savedir = os.path.join(work_dir,'csv_files/nnet_survival/'+str(n_features)+'_features/'+term_pred)
    try: 
        os.makedirs(savedir)
    except OSError:
        if not os.path.isdir(savedir):
            raise
    actual_fold = fold+1
    pred_surv.to_csv(savedir+'/pred_prob_surv_fold_'+str(actual_fold)+'.csv', index = None, header = True)


    ## SHAP:
    
    X_train_df_summary_50 = shap.kmeans(featurespace_train_df, 50)
    explainer_kmeans_50 = shap.KernelExplainer(f_nnet_input_as_matrix, X_train_df_summary_50, feature_names=feature_name_list) 
    kmeans_shaps_50 = explainer_kmeans_50.shap_values(featurespace_test_df)

    vimp_shap_kmeans_50 = vimp_shap_tabular(feature_name_list, np.abs(kmeans_shaps_50).mean(0))
    
    vimp_shap_kmeans_50.to_csv(savedir+'/shap_kernel_explainer_fold_'+str(actual_fold)+'.csv', index = None, header = True)

    


FOLD 0:

Train C-index fold 1 :
0.9738461104953694
Test C-index fold 1 :
0.6758348512446873




FOLD 1:

Train C-index fold 2 :
0.9818722993053403
Test C-index fold 2 :
0.8846991560032671




FOLD 2:

Train C-index fold 3 :
0.9758715212006411
Test C-index fold 3 :
0.7011612021857924




FOLD 3:

Train C-index fold 4 :
0.9579693378394382
Test C-index fold 4 :
0.4807823361046749




FOLD 4:

Train C-index fold 5 :
0.9631486519685899
Test C-index fold 5 :
0.673266648396821


...



FOLD 20:

Train C-index fold 21 :
0.975554369404802
Test C-index fold 21 :
0.6776687116564417




FOLD 21:

Train C-index fold 22 :
0.9866983351347424
Test C-index fold 22 :
0.8030637593154845




FOLD 22:

Train C-index fold 23 :
0.9675685979068902
Test C-index fold 23 :
0.565860400829302




FOLD 23:

Train C-index fold 24 :
0.9672632296161708
Test C-index fold 24 :
0.7594404630650496




FOLD 24:

Train C-index fold 25 :
0.961323226992302
Test C-index fold 25 :
0.8373081140350878


In [94]:
savedir

'U:/Hieu/CARDIA_project/CARDIA_project/cvd_outcome_rerun_2\\csv_files/nnet_survival/all_features/short_term'

In [92]:
## Using Explainer function instead of KernelExplainer:
# explainer_full_training = shap.Explainer(f_nnet, featurespace_train_df.iloc[:,:], feature_names=feature_name_list) 
# shaps_full_training_on_test_set = explainer_full_training(featurespace_test_df, max_evals = 2 * len(feature_name_list) + 1)
# vimp_shap_full_training_on_test_set = vimp_shap_tabular(feature_name_list, np.abs(shaps_full_training_on_test_set.values).mean(0))
# vimp_shap_full_training_on_test_set.iloc[:21,:]

In [93]:
## Summpary plot (or beeswarm plot):
# shap.summary_plot(kmeans_shaps_50,featurespace_test_df)